In [29]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

# Load the DataFrame
df = pd.read_csv("soilmoisture_dataset.csv", index_col=0)

# Remove empty rows
df = df.dropna()

# Reset the index without adding the old index as a column
df.reset_index(drop=True, inplace=True)

# Drop unnecessary columns
df = df.drop(['soil_temperature', 'datetime'], axis=1)

# Split 90% of the data into `train_val` and 10% into `test`
train_val, test = train_test_split(df, test_size=0.2, random_state=42)
X_cross_val = train_val.drop('soil_moisture', axis=1)
y_cross_val = train_val['soil_moisture']
X_test = test.drop('soil_moisture', axis=1)
y_test = test['soil_moisture']

# Define the adjusted R-squared function
def adjR2(r2, n_features, n_samples):
    return 1 - (1 - r2) * (n_samples - 1) / (n_samples - n_features - 1)

# Create a Lasso model with L1 regularization
lasso_model = Lasso(alpha=0.0001, random_state=42, max_iter=10000)  # Adjust alpha for regularization strength
# Create an SVR model for regression
svr_model = SVR(kernel='rbf', C=10000, epsilon=0.1)  # Adjust 'C' and 'epsilon' for different regularization strengths

# Perform Lasso for feature selection
lasso_model.fit(X_cross_val, y_cross_val)
coef = lasso_model.coef_

# Select the top 15 features based on the absolute value of the coefficients
top_k = 15
selected_indices = np.argsort(np.abs(coef))[-top_k:]  # Select the top 15 features
selected_columns = X_cross_val.columns[selected_indices].tolist()

print(f"Top 15 selected columns:", selected_columns)

# Initialize a list to store results
results = []

# Iterate through all subsets of the top 15 features (from 1 feature to 15 features)
for n in range(1, top_k + 1):
    current_features = selected_columns[:n]
    print(f"Evaluating {n} features: {current_features}")
    
    # Perform 10-fold cross-validation
    kf = KFold(n_splits=10, shuffle=True, random_state=42)

    rmse_list = []
    adjR_list = []

    for train_index, test_index in kf.split(X_cross_val):
        # Split the data into training and testing sets
        X_train, X_val = X_cross_val.iloc[train_index][current_features], X_cross_val.iloc[test_index][current_features]
        y_train, y_val = y_cross_val.iloc[train_index], y_cross_val.iloc[test_index]

        # Scale the features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)

        # Train the SVR model
        svr_model.fit(X_train_scaled, y_train)
        y_pred = svr_model.predict(X_val_scaled)

        # Calculate metrics
        mse = mean_squared_error(y_val, y_pred)
        rmse_list.append(sqrt(mse))

        r2 = r2_score(y_val, y_pred)
        adjR_list.append(adjR2(r2, X_val_scaled.shape[1], X_val_scaled.shape[0]))

    # Average RMSE and Adjusted R² across folds
    average_rmse = sum(rmse_list) / len(rmse_list)
    average_adjR = sum(adjR_list) / len(adjR_list)

    # Print results
    print(f"Features: {n}, Average RMSE: {round(average_rmse, 4)}, Average Adjusted R²: {round(average_adjR, 4)}")

    # Store results
    results.append({
        'Number of Features': n,
        'Selected Columns': ', '.join(current_features),
        'Average RMSE': round(average_rmse, 4),
        'Average Adjusted R²': round(average_adjR, 4)
    })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Save results to Excel
results_df.to_excel("Lasso_feature_combinations_results.xlsx", index=False)

print("Results saved to 'Lasso_feature_combinations_results.xlsx'")


c:\Users\alexk\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.204e+01, tolerance: 7.053e-01
  model = cd_fast.enet_coordinate_descent(


Top 15 selected columns: ['798', '498', '746', '454', '586', '574', '590', '510', '802', '582', '598', '506', '578', '462', '494']
Evaluating 1 features: ['798']
Features: 1, Average RMSE: 1.746, Average Adjusted R²: 0.7497
Evaluating 2 features: ['798', '498']
Features: 2, Average RMSE: 1.3665, Average Adjusted R²: 0.8413
Evaluating 3 features: ['798', '498', '746']
Features: 3, Average RMSE: 1.4603, Average Adjusted R²: 0.8097
Evaluating 4 features: ['798', '498', '746', '454']
Features: 4, Average RMSE: 1.1915, Average Adjusted R²: 0.8589
Evaluating 5 features: ['798', '498', '746', '454', '586']
Features: 5, Average RMSE: 1.0261, Average Adjusted R²: 0.8963
Evaluating 6 features: ['798', '498', '746', '454', '586', '574']
Features: 6, Average RMSE: 0.8788, Average Adjusted R²: 0.9227
Evaluating 7 features: ['798', '498', '746', '454', '586', '574', '590']
Features: 7, Average RMSE: 0.8703, Average Adjusted R²: 0.9223
Evaluating 8 features: ['798', '498', '746', '454', '586', '574',